In [1]:
#from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import warnings
import numpy as np
import pandas as pd

In [2]:
warnings.filterwarnings("ignore")
data = pd.read_csv('SeoulBike.csv',header = None)
data.columns = {'Date':int,
               'Rented_Bike_Count':int,
               'Hour':int,
               'Temperature':int,
                'Humidity':int,
               'Wind_speed':int,
               'Visibility':int,
               'Dew_point_temperature':int,
               'Solar_Radiation':int,
               'Rainfall':int,
               'Snowfall':int,
               'Seasons':'category',
               'Holiday':'category',
               'Functioning_Day':'category'}
data

,Date,Rented_Bike_Count,Hour,Temperature,Humidity,Wind_speed,Visibility,Dew_point_temperature,Solar_Radiation,Rainfall,Snowfall,Seasons,Holiday,Functioning_Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,30/11/2018,1003,19,4.2,34,2.6,1894,-10.3,0.0,0.0,0.0,Autumn,No Holiday,Yes
8756,30/11/2018,764,20,3.4,37,2.3,2000,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes
8757,30/11/2018,694,21,2.6,39,0.3,1968,-9.9,0.0,0.0,0.0,Autumn,No Holiday,Yes
8758,30/11/2018,712,22,2.1,41,1.0,1859,-9.8,0.0,0.0,0.0,Autumn,No Holiday,Yes


In [3]:
le=LabelEncoder()
for col in data[['Date', 'Rented_Bike_Count', 'Hour', 'Seasons', 'Holiday', 'Temperature', 'Functioning_Day', 'Humidity', 'Wind_speed', 'Visibility', 'Dew_point_temperature', 'Solar_Radiation', 'Rainfall', 'Snowfall']]:
    data[col]=le.fit_transform(data[col])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8760 entries, 0 to 8759
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype
---  ------                 --------------  -----
 0   Date                   8760 non-null   int32
 1   Rented_Bike_Count      8760 non-null   int64
 2   Hour                   8760 non-null   int64
 3   Temperature            8760 non-null   int64
 4   Humidity               8760 non-null   int64
 5   Wind_speed             8760 non-null   int64
 6   Visibility             8760 non-null   int64
 7   Dew_point_temperature  8760 non-null   int64
 8   Solar_Radiation        8760 non-null   int64
 9   Rainfall               8760 non-null   int64
 10  Snowfall               8760 non-null   int64
 11  Seasons                8760 non-null   int32
 12  Holiday                8760 non-null   int32
 13  Functioning_Day        8760 non-null   int32
dtypes: int32(4), int64(10)
memory usage: 821.4 KB


In [4]:
#查詢缺失值，並丟棄不需要的屬性欄位
data.replace(" ?", np.nan, inplace= True)
new_data = data.dropna()
print(r'總比數紀錄:{}'.format(data.shape[0]))
print(r'刪除含缺漏紀錄資料後比數:{}'.format(data.dropna().shape[0]))
pd.get_dummies(new_data['Date'])
Date_encoding = pd.get_dummies(new_data['Date'], prefix= 'Date')
new_data = new_data.drop('Date', 1)

pd.get_dummies(new_data['Rented_Bike_Count'])
Rented_Bike_Count_encoding = pd.get_dummies(new_data['Rented_Bike_Count'], prefix= 'Rented_Bike_Count')
#new_data = new_data.drop('Rented_Bike_Count', 1)

pd.get_dummies(new_data['Hour'])
Hour_encoding = pd.get_dummies(new_data['Hour'], prefix= 'Hour')
#new_data = new_data.drop('marital-status', 1)

pd.get_dummies(new_data['Temperature'])
Temperature_encoding = pd.get_dummies(new_data['Temperature'], prefix= 'Temperature')

pd.get_dummies(new_data['Humidity'])
Humidity_encoding = pd.get_dummies(new_data['Humidity'], prefix= 'Humidity')
new_data = new_data.drop('Humidity', 1)

pd.get_dummies(new_data['Wind_speed'])
Wind_speed_encoding = pd.get_dummies(new_data['Wind_speed'], prefix= 'Wind_speed')

pd.get_dummies(new_data['Visibility'])
Visibility_encoding = pd.get_dummies(new_data['Visibility'], prefix= 'Visibility')

pd.get_dummies(new_data['Dew_point_temperature'])
Dew_point_temperature_encoding = pd.get_dummies(new_data['Dew_point_temperature'], prefix= 'Dew_point_temperature')

pd.get_dummies(new_data['Solar_Radiation'])
Solar_Radiation_encoding = pd.get_dummies(new_data['Solar_Radiation'], prefix= 'Solar_Radiation')

pd.get_dummies(new_data['Rainfall'])
Rainfall_encoding = pd.get_dummies(new_data['Rainfall'], prefix= 'Rainfall')

pd.get_dummies(new_data['Snowfall'])
Snowfall_encoding = pd.get_dummies(new_data['Snowfall'], prefix= 'Snowfall')

pd.get_dummies(new_data['Seasons'])
Seasons_encoding = pd.get_dummies(new_data['Seasons'], prefix= 'Seasons')

pd.get_dummies(new_data['Holiday'])
Holiday_encoding = pd.get_dummies(new_data['Holiday'], prefix= 'Holiday')

pd.get_dummies(new_data['Functioning_Day'])
Functioning_Day_encoding = pd.get_dummies(new_data['Functioning_Day'], prefix= 'Functioning_Day')

#class_mapping = {' >50K': 1, ' <=50K': 0}
#new_data['listing_of_attributes'] = new_data['listing_of_attributes'].map(class_mapping)

pd.DataFrame(new_data)

總比數紀錄:8760
刪除含缺漏紀錄資料後比數:8760


,Rented_Bike_Count,Hour,Temperature,Wind_speed,Visibility,Dew_point_temperature,Solar_Radiation,Rainfall,Snowfall,Seasons,Holiday,Functioning_Day
0,253,0,111,22,1788,114,0,0,0,3,1,1
1,203,1,108,8,1788,114,0,0,0,3,1,1
2,172,2,103,10,1788,113,0,0,0,3,1,1
3,106,3,101,9,1788,114,0,0,0,3,1,1
4,77,4,103,23,1788,104,0,0,0,3,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
8755,991,19,205,26,1682,187,0,0,0,0,1,1
8756,755,20,197,23,1788,191,0,0,0,0,1,1
8757,686,21,189,3,1756,191,0,0,0,0,1,1
8758,704,22,184,10,1648,192,0,0,0,0,1,1


In [5]:
#合併 onehot encoding
new_data = pd.concat([new_data['Rented_Bike_Count'], Hour_encoding,Temperature_encoding,
           Wind_speed_encoding,Visibility_encoding,Dew_point_temperature_encoding, Solar_Radiation_encoding, Rainfall_encoding, Snowfall_encoding, Seasons_encoding, Holiday_encoding, Functioning_Day_encoding],axis=1)
new_data

,Rented_Bike_Count,Hour_0,Hour_1,Hour_2,Hour_3,Hour_4,Hour_5,Hour_6,Hour_7,Hour_8,...,Snowfall_49,Snowfall_50,Seasons_0,Seasons_1,Seasons_2,Seasons_3,Holiday_0,Holiday_1,Functioning_Day_0,Functioning_Day_1
0,253,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
1,203,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
2,172,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
3,106,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
4,77,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,991,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
8756,755,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
8757,686,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1
8758,704,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,1


In [6]:
train_data=new_data.sample(frac=0.8)
test_data = new_data[~new_data.index.isin(train_data.index)]

new_cols_train=list(new_data.columns)
new_cols_train.remove('Rented_Bike_Count')

new_cols_test=list(new_data.columns)
new_cols_test.remove('Rented_Bike_Count')

#x_train, x_test, y_train, y_test = train_test_split(new_cols_train, new_cols_test, test_size= 0.20)
x_train, y_train = train_data[new_cols_train].values, train_data['Rented_Bike_Count'].values
x_test, y_test = test_data[new_cols_test].values, test_data['Rented_Bike_Count'].values

In [7]:
x_train.shape
#print(y_train.shape)

(7008, 3445)

In [8]:
#KNN
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)
print("MAE:",mean_absolute_error(y_test,y_pred))
print("RMSE : %.4g" % mean_squared_error(y_test, y_pred,squared=False))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred))

MAE: 296.48801369863014
RMSE : 463.5
MAPE: 2.2093857760987108e+16


In [9]:
#XGBoost
xgbc = XGBRegressor()
xgbc.fit(x_train, y_train)
y_pred_xgb = xgbc.predict(x_test)
print("MAE:",mean_absolute_error(y_test,y_pred_xgb))
print("RMSE : %.4g" % mean_squared_error(y_test, y_pred_xgb,squared=False))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred_xgb))

MAE: 189.58957085652983
RMSE : 267.2
MAPE: 9226195280910102.0


In [10]:
#RandomForest
# 建立 Random Forest Classifier 模型
randomForestModel = RandomForestClassifier(n_estimators=100, criterion = 'gini')
randomForestModel.fit(x_train, y_train)
y_pred_rf = randomForestModel.predict(x_test)
print("MAE:",mean_absolute_error(y_test,y_pred_rf))
print("RMSE : %.4g" % mean_squared_error(y_test, y_pred_rf,squared=False))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred_rf))

MAE: 372.337899543379
RMSE : 547.5
MAPE: 1.173551865165731


In [11]:
#SVR
from sklearn.svm import LinearSVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
regr = make_pipeline(StandardScaler(), LinearSVR(random_state=0, tol=1e-5))
regr.fit(x_train, y_train)
y_pred_lsvr = regr.predict(x_test)
print("MAE:",mean_absolute_error(y_test,y_pred_lsvr))
print("RMSE : %.4g" % mean_squared_error(y_test, y_pred_lsvr,squared=False))
print("MAPE:",mean_absolute_percentage_error(y_test,y_pred_lsvr))

MAE: 293.1762851721479
RMSE : 376.7
MAPE: 3.993673546867353e+16
